In [5]:
import os
import sys

while os.path.split(os.getcwd())[1] != 'RecSysChallenge2023-Team':
    os.chdir('..')
sys.path.insert(1, os.getcwd())

import Daniele.Utils.MatrixManipulation as mm
import Daniele.Utils.MyDataManager as dm

from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.preprocessing import normalize
import similaripy
import scipy.sparse as sp
import numpy as np

URMv = dm.getURMviews()
URMo = dm.getURMopen()
# URM_all = mm.defaultExplicitURM(urmv=URMv, urmo=URMo, normalize=False, add_aug=False)
URM_all = URMo + URMv
URM_all.data = np.ones(len(URM_all.data))

ICMt = dm.getICMt()
ICMl = dm.getICMl()

URM_aug = sp.vstack([URM_all, ICMt.T])

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, [10])
evaluator_test = EvaluatorHoldout(URM_test, [10])

earlystopping_keywargs = {
    "validation_every_n": 10,
    "stop_on_validation": True,
    "evaluator_object": evaluator_validation,
    "lower_validations_allowed": 5,
    "validation_metric": 'MAP_MIN_DEN',
}

EvaluatorHoldout: Ignoring 791 ( 1.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 343 ( 0.8%) Users that have less than 1 test interactions


In [6]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

recommender = SLIM_BPR_Cython(URM_train=URM_train)
recommender.fit(
    epochs=200,
    symmetric=True,
    lambda_i=0.49782971629013434,
    lambda_j=0.09226465664615968,
    learning_rate=0.0020007963127774316,
    topK=201,
    **earlystopping_keywargs
)

SLIM_BPR_Recommender: URM Detected 3461 (12.4%) items with no interactions.
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.


/bin/sh: free: command not found


Processed 41629 (100.0%) in 1.55 sec. BPR loss is 1.28E-04. Sample per second: 26806
SLIM_BPR_Recommender: Epoch 1 of 200. Elapsed time 0.57 sec
Processed 41629 (100.0%) in 0.89 sec. BPR loss is 3.85E-04. Sample per second: 46758
SLIM_BPR_Recommender: Epoch 2 of 200. Elapsed time 0.91 sec
Processed 41629 (100.0%) in 1.17 sec. BPR loss is 6.56E-04. Sample per second: 35566
SLIM_BPR_Recommender: Epoch 3 of 200. Elapsed time 1.19 sec
Processed 41629 (100.0%) in 0.43 sec. BPR loss is 8.98E-04. Sample per second: 96390
SLIM_BPR_Recommender: Epoch 4 of 200. Elapsed time 1.45 sec
Processed 41629 (100.0%) in 0.67 sec. BPR loss is 1.13E-03. Sample per second: 61988
SLIM_BPR_Recommender: Epoch 5 of 200. Elapsed time 1.69 sec
Processed 41629 (100.0%) in 0.90 sec. BPR loss is 1.35E-03. Sample per second: 46077
SLIM_BPR_Recommender: Epoch 6 of 200. Elapsed time 1.92 sec
Processed 41629 (100.0%) in 1.15 sec. BPR loss is 1.53E-03. Sample per second: 36032
SLIM_BPR_Recommender: Epoch 7 of 200. Elapsed

In [3]:
result, result_string = evaluator_validation.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 40905 (100.0%) in 32.19 sec. Users per second: 1271


In [4]:
result

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.041149,0.066277,0.061546,0.018297,0.028978,0.132338,0.062205,0.049322,0.296247,0.154809,...,0.982608,0.291095,0.982608,0.019271,7.871521,0.988052,0.043616,0.58346,5.282048,0.308093


In [14]:
result_test, result_test_string = evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 73467 (100.0%) in 52.00 sec. Users per second: 1413


In [15]:
result_test

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.070565,0.147511,0.144668,0.046104,0.094054,0.290055,0.153246,0.09486,0.44978,0.365552,...,0.987738,0.444265,0.987738,0.490863,13.994219,1.000236,1.111415,1.039177,0.166488,0.537435
